In [1]:
import time
import numpy as np
from gep_config_parser import *
from data_wrangling import dataframe_to_dict
import pyomo.environ as pyo

from gep_main import run_model, prep_data, run_model_no_bounds
from gep_problem import GEPProblem

import torch

DTYPE = torch.float64

torch.set_default_dtype(DTYPE)

# DEVICE = (
#     torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
# )

DEVICE = "cpu"

Parsing the config file
Initializing the solver
Using Gurobi


In [2]:

CONFIG_FILE_NAME        = "config.toml"
VISUALIZATION_FILE_NAME = "visualization.toml"

HIGHS  = "HiGHS"
GUROBI = "Gurobi"

SAMPLE_DURATION = 12 # 12 hours

## Step 0: Activate environment - ensure consistency accross computers
# print("Reading the data")
# print("Activating the environment")


In [3]:

## Step 1: parse the input data
print("Parsing the config file")

data = parse_config(CONFIG_FILE_NAME)
experiment = data["experiment"]
outputs_config = data["outputs_config"]

print("Initializing the solver")
optimizer_name = data["optimizer_config"]["solver"]

# Determine the optimizer
if optimizer_name == HIGHS:
    raise NotImplementedError(f"{optimizer_name}: Not implemented")
elif optimizer_name == GUROBI:
    
    print(f"Using {GUROBI}")
    optimizer = "gurobi_direct"
else:
    raise ValueError(f"{optimizer_name}: Not implemented")

Parsing the config file
Initializing the solver
Using Gurobi


In [4]:
for i, experiment_instance in enumerate(experiment["experiments"]):
    # Setup output dataframe
    df_res = pd.DataFrame(columns=["setup_time", "presolve_time", "barrier_time", "crossover_time", "restore_time", "objective_value"])

    input_T, input_N, input_G, input_L, input_pDemand, input_pGenAva, input_pVOLL, input_pWeight, input_pRamping, input_pInvCost, input_pVarCost, input_pUnitCap, input_pExpCap, input_pImpCap = prep_data(experiment_instance)

    T_ranges = [range(i, i + SAMPLE_DURATION, 1) for i in range(1, len(input_T), SAMPLE_DURATION)]
    objective_values = []
    times = []
    models = []
    solvers = []
    for t in T_ranges:
        # Run one experiment for j repeats
        model, solver, time_taken = run_model_no_bounds(experiment_instance, t, input_N, input_G, input_L, input_pDemand, input_pGenAva, input_pVOLL, input_pWeight, input_pRamping, input_pInvCost, input_pVarCost, input_pUnitCap, input_pExpCap, input_pImpCap)
        models.append(model)
        solvers.append(solver)

Wrangling the input data
Populating the model
Adding model variables
Formulating the objective
Adding model constraints
Solving the optimization problem
Set parameter OutputFlag to value 1
Set parameter LogFile to value "outputs/Gurobi/output.txt"
Set parameter Crossover to value 1
Set parameter FeasibilityTol to value 1e-09
Set parameter QCPDual to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 22.3.0 22D49)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
FeasibilityTol  1e-09
Method  2
Crossover  1
QCPDual  1

Optimize a model with 323 rows, 113 columns and 650 nonzeros
Model fingerprint: 0x1173bacc
Coefficient statistics:
  Matrix range     [7e-02, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 3e+04]
Presolve removed 309 rows and 104 columns
Presolve time: 0.00s
Presolved: 14 rows, 9 columns, 42 nonzeros

Ordering tim

In [5]:
#T, N, G, L, pDemand, pGenAva, pVOLL, pWeight, pRamping, pInvCost, pVarCost, pUnitCap, pExpCap, pImpCap
data = GEPProblem(input_T, input_N, input_G, input_L, input_pDemand, input_pGenAva, input_pVOLL, input_pWeight, input_pRamping, input_pInvCost, input_pVarCost, input_pUnitCap, input_pExpCap, input_pImpCap, sample_duration=SAMPLE_DURATION, shuffle=False)

# X contains [pGenAva, pDemand]
all_X = torch.concat([data.trainX, data.validX, data.testX])

Size of train set: 584
Size of val set: 73
Size of test set: 73
Size of mu: 285
Size of lambda: 36
Number of variables (size of y): 111
Number of inputs (size of X): 72


In [6]:
from get_gurobi_vars import OptValueExtractor

extractor = OptValueExtractor()
for model in models:
    extractor.extract_values(model)

In [8]:
rhs_eq = data.rhs_eq
rhs_ineq = data.rhs_ineq
mu = torch.stack([extractor.opt_targets[idx]["mu"] for idx in range(len(extractor.opt_targets))])
lamb = torch.stack([extractor.opt_targets[idx]["lamb"] for idx in range(len(extractor.opt_targets))])


dual_obj_vals = data.dual_obj_fn(mu, lamb, rhs_eq, rhs_ineq)

objectives = [model.obj() for model in models]

assert np.allclose(dual_obj_vals, objectives, 1e-6)
